### Site parsing with saving to sql server. simple example for wiki banking list

In [1]:
#github#please, insert you sql server parameters here
server = 'yourservername' 
database = 'AdventureWorks' 
username = 'username' 
password = 'yourpassword'  

In [2]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import re
import json
import pyodbc

#### Procedure to extract list from url-site-body

In [11]:
# Function to generate Exception-text
def extract_list_exception(url, subject):
    return print('EXCEPTION!! Body of ', subject,' is empty; Procedure extract_list(url =', url ,' ...')

#rules[0]: body/tbody/None
#rules[1]: div/table
#rules[2]: class = 
#rules[3]: tb/th/
#rules[4]: tb/th/
#rules[5]: start position  = 0 (if not Header), = 1 (if Header)
#rules[6]: finish position  = None (if not Total), = 1 (if Total)
# Function to parse url by rules. Marker used for an algorithm branches.


def extract_list (url, rules, marker = None, header = False, total = False, debug=False, area_number=0):
    entities = []
    hrefs = []
    
    if url != '#':
        response = requests.get(url)    
        if response.status_code != 200:
            print('Web site does not exist')
        else:

            obj = BeautifulSoup(response.content, features="lxml")

          #  if rules[0] == 'body':
          #      area = obj.body.find(rules[1],{'class':rules[2]})
          #  if rules[0] == 'tbody':
          #      area = obj.tbody.find(rules[1],{'class':rules[2]})    
            if rules[0] in ('None',''):
                area = obj.find_all(rules[1],{'class':rules[2]}) 
                entities = []
                hrefs = []

            if debug == 2:
                print(area)

            if area in (None,[]):
                extract_list_exception(url,'area')

            else:
                elements = area[area_number].find_all(rules[3])  

                if elements  in (None,[]):
                    extract_list_exception(url,'elements')        


                else: 
                    if debug == True:
                        print(elements[0])   #header?
                        print(elements[1])                 
                        print(elements[-2])                
                        print(elements[-1])  #total?

                    start_pos = int(header)
                    if total != False:
                        finish_pos = -int(total)
                    else:
                        finish_pos = None

                    if debug == True:
                        print(start_pos,finish_pos)

                    if rules[5] == '':
                        entities = [el.find(rules[4]).text.strip()  for el in elements[start_pos:finish_pos] ]
                        hrefs = [el.find(rules[4]).get('href') for el in elements[start_pos:finish_pos]  ]
                    else:    
                        entities = [el.find(rules[4]).find(rules[5]).text.strip()  for el in elements[start_pos:finish_pos]  ] 
                        hrefs = [el.find(rules[4]).find(rules[5]).get('href') for el in elements[start_pos:finish_pos]  ]

                    if debug == True:
                        print(entities,'entities')
                        print(hrefs,'hrefs')

                   
    return  pd.DataFrame(zip(entities,hrefs), columns = ['entity','href'])

#### step 1. Extract bank list and bank links from wiki

In [12]:
url = 'https://en.wikipedia.org/wiki/List_of_banks_in_Serbia'
 
banks_wiki = extract_list(url,rules=['','table',"wikitable sortable",'tr','td','a'],header=True,total=True)
pref_wiki = 'https://en.wikipedia.org'
banks_wiki['href_wiki'] = [pref_wiki+str(x) for x in banks_wiki.href]
banks_wiki['website'] = banks_wiki.href_wiki.apply(lambda url_h: extract_list(url=url_h,rules=['','table',"infobox vcard",'tr','a',''],header=-1,debug=False).href[0] 
                                            if 'index.php?'  not in url_h 
                                            else ''  )
#banks_wiki

In [13]:
banks_wiki.head()

,entity,href,href_wiki,website
0,3 banka,/w/index.php?title=3_banka&action=edit&redlink=1,https://en.wikipedia.org/w/index.php?title=3_b...,
1,Addiko Bank a.d.,/w/index.php?title=Addiko_Bank_a.d.&action=edi...,https://en.wikipedia.org/w/index.php?title=Add...,
2,AIK Banka,/wiki/AIK_Banka,https://en.wikipedia.org/wiki/AIK_Banka,http://www.aikbanka.rs/
3,Alta banka,/wiki/Alta_banka_a.d.,https://en.wikipedia.org/wiki/Alta_banka_a.d.,https://altabanka.rs/
4,API Bank a.d.,/w/index.php?title=API_Bank_a.d.&action=edit&r...,https://en.wikipedia.org/w/index.php?title=API...,


#### step 2. Save table to sql server

In [ ]:
#SQL# create table LepaPythonBase.dbo.WikiBanks (entity char(100),href char(100),href_wiki char(100),website  char(100))

In [16]:
server = 'LENOVO-PC41' 
database = 'LepaPythonBase' 
username = 'Lepa' 
password = '4eptember' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

# Insert Dataframe into SQL Server:
for index, row in banks_wiki.iterrows():
     cursor.execute("INSERT INTO dbo.WikiBanks (entity,href,href_wiki,website) values(?,?,?,?)", row.entity, row.href, row.href_wiki, row.website)
cnxn.commit()
cursor.close()